# Constraint Satisfaction Problems

This notebook presents, in an interactive way, the lecture notes of MIT's 16.413 course **Principles of Autonomy and Decision Making** for the chapter on **Constraint Satisfaction**.

The material below is highly influenced as well by **Chapter 6 Constraint Satisfaction Problems** of the book *Artificial Intelligence: A Modern Approach*.

Let us first import necessary modules and helper functions. Run the cell below:

In [1]:
from __future__ import print_function
from lib.notebook import psource, pseudocode, plot_NQueens
from lib.n_queens_csp import NQueensCSP
from lib.backtracking import backtracking_search, backtrack
import time
%matplotlib inline

# Hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")


ModuleNotFoundError: No module named 'utils'

# CONTENTS

 1. **Introduction to Constraint Propagation Problems (L13 & L14)**:
    1. *Definition*
    2. *Examples*

 2. **Solving CSPs (L14 & L 15)**:
    1. *Constraint Propagation in CSPs*:
        1. Arc-Consistency:
            1. Directed arc consistency
            2. Full arc consistency

        2. Propagation:
            1. AC-1:
                * Definition
                * Example: solving Sudoku with AC-1
                * Complexity Analysis
                * Soundness and Completeness
            2. AC-3:
                * Definition
                * Example: solving Sudoku with AC-3
                * Complexity Analysis
                * Soundness and Completeness
        3. Numerical constraints:
            1. Propagation for Integer Domains
            2. Pruning Integer Domains
        4. Global constraints:
            1. AllDiff
            2. Example: global vs local constraints NQueens
    2. *[Search in CSPs](#search_in_csps)*:
        1. [Generic Search](#generic_search):
            * Formulating CSPs as a state space search problem
            * Complexity
            * Example: NQueens with generic search (5 Queens)
        2. [Backtracking Search (BT)](#backtracking):
            * Definition
            * Complexity
            * Example: solving NQueens (15 Queens)
        3. [Backtracking Search with Forward Checking (BT-FC)](#backtracking_w_fc):
            * Definition
            * Complexity
            * Example: solving NQueens (30 Queens)              
        4. [**Backtracking while Maintaining Arc Consistency (BT-MAC)**](#backtracking_w_mac):
            * Definition
            * Complexity
            * Example: solving NQueens (X Queens)
        4. BT-DC with Dynamic Variable Ordering:
            * Definition
            * Complexity
            * Example: solving NQueens (1000 Queens)

    3. *Elimination for Constraints in CSPs*:
        1. Variable Elimination for Constraints:
            * Definition: join and project
            * Example
        2. Bucket elimination (Adaptive Consistency):
            * Definition
            * Example

 3. **Appendix**:
    1. Constraint Optimization Problem:
        1. Branch-and-Bound
        2. Example: using Map Coloring problem with cost added to the colors to use.
    2. Conflict-directed Back Jumping (CBJ)
        * Definition
        * Example
    3. Incremental Repair / Iterative Repair / Min-Conflict Heuristic
        * Definition
        * Example

## Introduction 

Constraint satisfaction problem represents states using a list of variables with assignments and
conditions for a solution in the form of constraints on the variables. This allows us to treat
the state space with structure - using a factored representation for each state contrary to the
black box view used in state space search problems.


Formally, Constraint Satisfaction Problem (CSP) can be defined as a triple of < X, D, C > where
X is a set of variables, D is the domain of each of the variables in V and C is the set of constraints
defining the problem. Each constraint in C can be thought of as a pair of scope S which is a
subset of V and a relation R between variables in S. The problem of having seeking a set with one
A and two B’s can be cast in the CSP framework with V = {A, B}, D = {1, 2} and constraint
C = {[{A, B}, {{1, 2}, {1, 1}}], [{A, B}, {{1, 2}, {2, 2}}]} where the first constraint tells that there
must be exactly one A and the second constraint imposes the fact that there must be two B’s. The
solution to the problem is when both the constraints are satisfied which is possible only when the
assignment is (1,2).

CSPs can be visualized using constraint graphs where the nodes represent the variables and the
edges represent the constraint between the variables connected by it. Any CSP can be represented
using a constraint graph because any constraint involving multiple variables can be converted to
binary variables by adding additional variables to the problem.

## Constraint Propagation

Apart from searching the state space, constraint programming also allows us to perform specific
type of inference called constraint Propagation which helps in eliminating unwanted parts of the
state space based on the constraint satisfaction.

A single variable in a CSP is said to be node-consistent if all it’s unary constraints are satisfied.
We can use this to delete some values in the domain of the variable that doesn’t satisfy the unary
constraint on that variable. A network is node-consistent if all the variables in it are also nodeconstraint.

A variable in CSP is said to be arc-consistent if every value in its domain satisfies binary constraints
imposed on the variable. Similar to node consistency, checking for arc-consistency can help reduce the domain size of a variable before performing search. Algorithm 1 shown below describes a Revise procedure that ensures that a given node Xi is arc-consistent.

<img src="Algorithm_1.png">

## AC-1 and AC-3

AC-1 is a constraint propagation algorithm that repeatedly checks for arc-consistency between two nodes and removes those values in the domain of each of the two variables that are not consistent. The algorithm is described below:

<img src="Algorithm_2.png">

In AC-1, the domain of a variable is changed by deleting the values in the variable’s domain that are arc inconsistent with another variable. We notice that everytime the domain of the variable shrinks, some equivalent values in other variables now become inconsistent. AC-1 doesn’t take advantage of this and hence it must be run iteratively until the domain of all the variables don’t change. AC-3, on the contrary, tries to take advantage of this fact by using a queue that has information about the next arcs to be checked based on the modifications made in the current iteration. This leads to a much faster constraint propagation. The algorithm is described below

<img src="Algorithm_3.png">

## <a name="search_in_csps"></a>2. Search in CSPs

Constraint propagation alone is not sufficient to solve many CSPs. In this case, a solution must be found using search algorithms.

In this section, we will explain different search algorithms used to solve CSPs.
The section is organized as follows:

1. [Generic Search](#generic_search):
    * Formulating CSPs as a state space search problem
    * Complexity
    * Example: NQueens with generic search (5 Queens)
2. [Backtracking Search (BT)](#backtracking):
    * Definition
    * Complexity
    * Example: solving NQueens (15 Queens)
3. [Backtracking Search with Forward Checking (BT-FC)](#backtracking_w_fc):
    * Definition
    * Complexity
    * Example: solving NQueens (30 Queens)              
4. [**Backtracking while Maintaining Arc Consistency (BT-MAC)**](#backtracking_w_mac):
    * Definition
    * Complexity
    * Example: solving NQueens (X Queens)
4. BT-DC with Dynamic Variable Ordering:
    * Definition
    * Complexity
    * Example: solving NQueens (1000 Queens)

<div class="alert alert-block alert-danger">
<b>TODO:</b> Provide example, slide 79 seems to show a case where arc-consistency alone does not solve the problem (but I think that path consistency would do constraint propagation up to the solution...). Can we find a sudoku example where we cannot rule out all possibilities by doing constraint propagation?
</div>

### <a name="generic_search"></a> a. Generic Search

#### Formulating CSPs as a state space search problem

CSPs can be formulated using the State Space search representation.
In particular, we can express a CSP as follows:
* **State**:
    - We can define as state a **partial assignment** to variables made thus far.
    For example, if we have variables $A$, $B$ and $C$ whith respective domains $[a_1, a_2]$, $[b_1, b_2]$, and  $[c_1, c_2]$ a partial assignment would be for example $\{A=a_2, C=c_1\}$, another one could be $\{B=b_2, C=c_2\}$. In contrast, we consider a **complete assignment** one where all variables would be assigned, such as $\{A=a_2, B=b_1, C=c_1\}$. Note that a complete assignment is still a (kind-of) partial assignment.
* **Initial State**:
    - Initially the state would be **no assignment**, which we can represent as the empty set $\{\}$.
* **Operator**:
    - We also need to define an action/operator to frame a CSP into a State Space search problem.
    In our case, this would correspond to the fact of **adding a new assignment**.
    In other words, our operator corresponds to the action of assigning a value to a yet unassigned variable $X=x$, which expands the State by one element. For example, if we start from the initial state $\{\}$, and we proceed with the action $X=x,$ we end up with a new state $\{X=x\}$, which is itself a partial assignment.
* **Goal Test**:
    - As in all State Space search problems, we need to define what is our goal state.
    In our case, this is achieved when all variables are assigned (meaning we have a **complete assignment**) and all constraints are satisfied (which we refer to as a **consistent assignment**).

Once the problem is formulated as a State Space search problem, we can use standard search algorithms such as:
* Breadth first search
* Depth first search
* Depth-limited search

to solve the given problem.

#### Complexity

Unfortunately, given a CSP with $n$ variables with a domain size of $d$, results in a branching factor at the top level of $nd$.
This is because any of the $n$ variables can be assigned any of $d$ values.
Once a variable has been assigned a value, we are still left with $n − 1$ variables which can be assigned $d$ values. By induction, we end up with a tree having $n!d^n ∼ O((nd)^n)$ leaves, which is even larger than the possible $d^n$ complete assignments!


Nevertheless, we can observe than in a CSP **the order when assigning variables does not influence the solution**. This property is known as **commutativity**. More specifically, a problem is commutative if the order of application of any given set of actions does no effect on the outcome. Therefore, there is no need to consider all variables at each level of the search tree, but just one at a time. 

This makes sense intuitvely, for example, if we consider the coloring map problem, we might decide whether to color a certain region red or blue, but we would not decide between coloring one region red or another region blue. Therefore, the number of leaves is reduced to $d^n$, where we just have to decide on an assignment for each variable at a time.

#### Example: NQueens with generic search (5 Queens)

Let us apply a generic search algorithm to the NQueens problem.
<div class="alert alert-block alert-danger">
<b>TODO:</b> Use generic search to solve NQueens
</div>

### <a name="backtracking"></a> b. Backtracking

Backtracking search suits the commutative property of CSPs, as it consists on a depth-first search that chooses one variable at a time. 

Moreover, it makes use of another property of CSPs that is **extensions to inconsistent partial assignments are always inconsistent**.
The backtracking search **backtracks** when an inconsistent partial assignment is reached. 

We show below the actual backtracking search algorithm, as presented in [5]. 

<a name="backtracking_algorithm"></a>
<img src="images/backtracking_algorithm.png" alt="drawing" width="600">

In plain words, the algorithm first chooses an unassigned variable, and loops over its domain by picking one value at
a time. Every time a value is taken, if it is consistent, then the algorithm continues to look for a
solution. If it is not consistent, then the algorithm backtracks by trying with another value instead.

The backtracking algorithm presented above makes use of a set of sub-functions that we detail
below:
* **Select-Unassigned-Variable(csp, assignment)**:
    - Decides which variable should be assigned next.
* **Order-Domain-Values(var, assignment, csp)**:
    - Decides the order in which the values of the variable should be tried. We will see in section 2.3.4 how modifications of this and the previous function result in different general purpose heuristics.
* **Inference(csp, var, assignment)**:
    - Every time a variable has been assigned a value, there is the opportunity to further reduce the domains of the rest of unassigned variables using inference, such as checking for arcconsistency. While the call to this function is not strictly necessary, we will see in section 2.3.3 that interleaving search and inference results in a more efficient algorithm.
    
The source code to implement this pseudo-code is actually fairly close to the actual implementation.
Below the actual source code in python:

In [ ]:
psource(backtracking_search)

In [ ]:
psource(backtrack)

<div class="alert alert-block alert-danger">
<b>TODO:</b> Implement algorithm in notebook and play with different implementations of the subfunctions. Show practical examples for Sudoku example
</div>

### <a name="backtracking_w_fc"></a> c. Backtracking with Forward Checking

By modifying the call to the function _Inference_ in the backtracking algorithm presented [above](#backtracking_algorithm), we can interleave search with inference to reduce the domain of yet unassigned variables, thereby reducing the search space. 

A simple form of inference is **forward checking**, which consists in establishing arc-consistency for the recently assigned variable with respect to all connected, yet unassigned, variables.

<div class="alert alert-block alert-danger">
<b>TODO:</b> Add at this point an example of forward-checking in sudoku problems
</div>

### <a name="backtracking_w_mac"></a>d. Backtracking while Maintaining Arc Consistency (BT-MAC)

#### Definition
<div class="alert alert-block alert-danger">
<b>TODO:</b> Definition
</div>

#### Complexity
<div class="alert alert-block alert-danger">
<b>TODO:</b> Complexity
</div>

#### Example: solving NQueens (X Queens)
<div class="alert alert-block alert-danger">
<b>TODO:</b> Example: solving NQueens (X Queens)
</div>
<div class="alert alert-block alert-danger">
<b>TODO:</b> What is the actual number for X above, aka how many Queens can we solve with MAC?
</div>

 ## Dynamic Variable Ordering
 
The function Select-Unassigned-Variable in algorithm 4 could be trivially set to pick the next unassigned variable in order, or alternatively pick a random variable each time. Nevertheless, these strategies rarely result in an efficient search. <font color='red'> TODO: Code an example where this is the case.</font> The Most Constrained Variable (MCV) heuristic. It consists in taking as next variable the one having the smallest domain. It is therefore also known as the minimum-remaining-values heuristic, or the fail-first heuristic. The advantage of using MCV is clear when we have a variable that has no legal values left, in this case, the MRV will select this variable first and failure will be detected immediately, which avoids further searches through other variables.

The MCV heuristic usually performs better than random or static ordering, sometimes by a factor
of 1000 or more, although the results vary widely depending on the problem. <font color='red'> TODO: Can we
show this using the Sudoku implementation? </font> We can also modify the function Order-Domain-Values in algorithm 4 to reach a solution faster. The Least Constraining Value (LCV) heuristic consists in taking as next value for the variable in hand the one that reduces by the least amount the domains of the neighbor variables. <font color='red'> TODO: Give Sudoku example or map coloring. </font> The heuristic is therefore just trying to leave the maximum flexibility for subsequent variable assignments. Note that if our objective is to list all possible solutions, this heuristic does not bring any advantage.

## Bucket Elimination

The idea of join and project used to eliminate variables when solving system of linear equation can be extended to constraints. This is made possible by bucket elimination. 

$\textbf{Join operation}$ for constraints is shown below:
<img src="pic_1.png">

$\textbf{Project operation}:$ C2 = $\pi_{2}C_{21}$
<img src="pic_2.png">

Important ideas behind bucket elimination are as follows:

* Eliminate: Derive all consequences that depend on each Vi
    by joining all constraints involving Vi

* Solve: Encode all solutions to constraints $\phi$,
    by systematically eliminating all variables.

* Generate: all solutions without backtrack search, by propagating.

In [ ]:
# DELETE once we finish this notebook
%load_ext autoreload
%autoreload 2

In [ ]:
# Solve n queens.
n_queens = NQueensCSP(8)

# Backtracking
start_time = time.time()
solution = backtracking_search(n_queens, fc=False)
end_time = time.time()
print(solution)
print(end_time - start_time)
n_queens.display(solution)

# Backtracking with forward checking
start_time = time.time()
solution = backtracking_search(n_queens, fc=True)
end_time = time.time()
print(solution)
print(end_time - start_time)
n_queens.display(solution)

plot_NQueens(solution)


### Helper Functions

We will now implement a few helper functions that will allow us to visualize the Coloring Problem; we'll also make a few modifications to the existing classes and functions for additional record keeping. To begin, we modify the **assign** and **unassign** methods in the **CSP** in order to add a copy of the assignment to the **assignment_history**. We name this new class as **InstruCSP**; it will allow us to see how the assignment evolves over time. 

In [ ]:
import copy
class InstruCSP(NQueensCSP):
    
    def __init__(self, N):
        super().__init__(N)
        self.assignment_history = []
        
    def assign(self, var, val, assignment):
        super().assign(var, val, assignment)
        self.assignment_history.append(copy.deepcopy(assignment))
    
    def unassign(self, var, assignment):
        super().unassign(var,assignment)
        self.assignment_history.append(copy.deepcopy(assignment))

Next, we define **make_instru** which takes an instance of **CSP** and returns an instance of **InstruCSP**.

In [ ]:
def make_instru(csp):
    return InstruCSP(len(csp.vars))

## N-QUEENS VISUALIZATION

Just like the Graph Coloring Problem, we will start with defining a few helper functions to help us visualize the assignments as they evolve over time. The **make_plot_board_step_function** behaves similar to the **make_update_step_function** introduced earlier. It initializes a chess board in the form of a 2D grid with alternating 0s and 1s. This is used by **plot_board_step** function which draws the board using matplotlib and adds queens to it. This function also calls the **label_queen_conflicts** which modifies the grid placing a 3 in any position where there is a conflict.

In [ ]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
import time
import ipywidgets as widgets
from IPython.display import display

In [ ]:
def label_queen_conflicts(assignment,grid):
    ''' Mark grid with queens that are under conflict. '''
    for col, row in assignment.items(): # check each queen for conflict
        conflicts = {temp_col: temp_row for temp_col,temp_row in assignment.items() if (temp_row == row and temp_col != col or (temp_row+temp_col == row+col and temp_col != col) or (temp_row-temp_col == row-col and temp_col != col))}
        
        # Place a 3 in positions where this is a conflict
        for col, row in conflicts.items():
                grid[col][row] = 3

    return grid

def make_plot_board_step_function(instru_csp):
    '''ipywidgets interactive function supports
       single parameter as input. This function
       creates and return such a function by taking
       in input other parameters.
    '''
    n = len(instru_csp.vars) 
    
    def plot_board_step(iteration):
        ''' Add Queens to the Board.'''
        data = instru_csp.assignment_history[iteration]
        
        grid = [[(col+row+1)%2 for col in range(n)] for row in range(n)]
        grid = label_queen_conflicts(data, grid) # Update grid with conflict labels.
        
        # color map of fixed colors
        cmap = matplotlib.colors.ListedColormap(['white','lightsteelblue','red'])
        bounds=[0,1,2,3] # 0 for white 1 for black 2 onwards for conflict labels (red).
        norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
        
        fig = plt.imshow(grid, interpolation='nearest', cmap = cmap,norm=norm)

        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)

        # Place the Queens Unicode Symbol
        for col, row in data.items():
            fig.axes.text(row, col, u"\u265B", va='center', ha='center', family='Dejavu Sans', fontsize=32)
        plt.show()
    
    return plot_board_step

def make_visualize(slider):
    ''' Takes an input a slider and returns 
        callback function for timer and animation
    '''
    
    def visualize_callback(Visualize, time_step):
        if Visualize is True:
            for i in range(slider.min, slider.max + 1):
                slider.value = i
                time.sleep(float(time_step))
    
    return visualize_callback
    

In [ ]:
twelve_queens_csp = NQueensCSP(12)
backtracking_instru_queen = make_instru(twelve_queens_csp)
result = backtracking_search(backtracking_instru_queen)

In [ ]:
backtrack_queen_step = make_plot_board_step_function(backtracking_instru_queen) # Step Function for Widgets

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8.0, 8.0)
matplotlib.rcParams['font.family'].append(u'Dejavu Sans')

iteration_slider = widgets.IntSlider(min=0, max=len(backtracking_instru_queen.assignment_history)-1, step=1, value=0)
w=widgets.interactive(backtrack_queen_step,iteration=iteration_slider)
display(w)

visualize_callback = make_visualize(iteration_slider)

visualize_button = widgets.ToggleButton(description = "Visualize", value = False)
time_select = widgets.ToggleButtons(description='Extra Delay:', options=['0', '0.1', '0.2', '0.5', '0.7', '1.0'])

a = widgets.interactive(visualize_callback, Visualize = visualize_button, time_step = time_select)
display(a)

